# **딥러닝(Deep Learning)**



---



# **인공신경망과 심층신경망**

## **1.인공신경망 : 퍼셉트론(Perceptron)**
- 참고
    - http://neuralnetworksanddeeplearning.com/chap1.html
    - http://neuralnetworksanddeeplearning.com/chap2.html

- **퍼셉트론(Perceptron)**
    - 1957년 프랑크 로젠블랫(Frank Rosenblatt)에 의해 고안된 초**기 형태의 인공 신경망**
    - **다수의 입력을 받아 하나의 출력을 내보내는 선형 분류기**
    - 각 입력에는 가중치가 곱해지고, 이들의 합이 특정 임계값을 넘으면 1을, 그렇지 않으면 0(또는 -1)을 출력
    - 단층 퍼셉트론은 선형적으로 분리 가능한 문제(예: AND, OR 게이트)만 해결할 수 있고, 비선형 문제(예: XOR 게이트)는 해결하지 못하는 한계가 있음

- **활성화 함수 (Activation Function)**
    - 뉴런의 최종 출력값을 결정하는 함수로, 입력 신호의 총합을 받아 다음 층으로 전달할지 여부와 그 강도를 조절
    - 비선형 활성화 함수를 사용해야 신경망이 **비선형적인 패턴을 학습**할 수 있음



- 퍼셉트론 수식 요약

| 항목           | 수식                                                                 |
|----------------|----------------------------------------------------------------------|
| **가중합**     | $  z = \mathbf{w} \cdot \mathbf{x} + b$
| **출력 함수**  | $( \hat{y} = f(z) )$, 여기서 f는 step function
| **가중치 업데이트** | \( w_i := w_i + $\eta$ (y - $\hat{y}$) x_i )                          |
| **바이어스 업데이트** | \( b := b + $\eta$ (y - $\hat{y}$) \)   

### **1) 퍼셉트론 기반 논리 게이트**

In [ ]:
import numpy as np

# 퍼셉트론 구현 함수
def perceptron(x1, x2, w1, w2, bias):
    threshold = 0.0
    tmp = x1 * w1 + x2 * w2 + bias
    return 1 if tmp > threshold else 0   # 임계치 값 설정

# 논리 게이트 정의
def AND(x1, x2):
    return perceptron(x1, x2, w1=0.5, w2=0.5, bias=-0.7)

def OR(x1, x2):
    return perceptron(x1, x2, w1=0.5, w2=0.5, bias=-0.2)

def NAND(x1, x2):
    return perceptron(x1, x2, w1=-0.5, w2=-0.5, bias=0.7)

def NOR(x1, x2):
    return perceptron(x1, x2, w1=-0.5, w2=-0.5, bias=0.2)

# 입력 조합
X = [(0, 0), (0, 1), (1, 0), (1, 1)]

# 출력 확인
print("x1 x2 | AND OR NAND NOR")
print("-------------------------")
for x1, x2 in X:
    print(f" {x1}  {x2} |  {AND(x1,x2)}   {OR(x1,x2)}    {NAND(x1,x2)}    {NOR(x1,x2)}")


### **2) (단층)퍼셉트론** (AND 게이트 예측 : 이진분류)

In [ ]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score

# --- 활성화 함수 및 손실 함수 정의 ---

def sigmoid(x):
    """시그모이드 함수"""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def sigmoid_derivative(x_activated):
    """시그모이드 함수의 미분 (x_activated는 시그모이드 통과한 값)"""
    return x_activated * (1 - x_activated)

def softmax(x):
    """소프트맥스 함수 (수치적 안정성 고려)"""
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

def mse_loss(y_true, y_pred):
    """평균 제곱 오차 계산"""
    return np.mean((y_true - y_pred)**2)

def cross_entropy_loss(y_true_one_hot, y_pred_softmax):
    """교차 엔트로피 손실 함수 (다중 클래스)"""
    # log(0) 방지를 위한 클리핑
    y_pred_softmax = np.clip(y_pred_softmax, 1e-12, 1. - 1e-12)
    num_samples = y_true_one_hot.shape[0]
    loss = -np.sum(y_true_one_hot * np.log(y_pred_softmax)) / num_samples
    return loss


# --- 단층 퍼셉트론(SLP) 클래스 ---
class SLP:
    def __init__(self, input_size, output_size, seed=None):
        if seed is not None: np.random.seed(seed)
        self.W = np.random.uniform(-1, 1, (input_size, output_size))
        self.b = np.zeros((1, output_size))
        self.z, self.a = None, None

    def forward(self, X):
        self.z = np.dot(X, self.W) + self.b
        self.a = sigmoid(self.z) # SLP는 주로 이진 분류에 sigmoid 사용
        return self.a

    def train(self, X, y_true, epochs, learning_rate, verbose_interval=None):
        losses = []
        num_samples = X.shape[0]

        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = mse_loss(y_true, y_pred)
            losses.append(loss)

            delta_activated = (y_pred - y_true) * sigmoid_derivative(y_pred)
            dW = (1/num_samples) * np.dot(X.T, delta_activated)
            db = (1/num_samples) * np.sum(delta_activated, axis=0, keepdims=True)
            self.W -= learning_rate * dW
            self.b -= learning_rate * db

            if verbose_interval and (epoch + 1) % verbose_interval == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.6f}")
        if verbose_interval and epochs % verbose_interval != 0:
             print(f"Epoch {epochs}/{epochs}, Loss: {losses[-1]:.6f}")
        elif not verbose_interval:
            print(f"Epoch {epochs}/{epochs}, Loss: {losses[-1]:.6f}")
        return losses

    def predict(self, X):
        return self.forward(X)


In [ ]:
# --- 메인 실행 부분 ---
if __name__ == '__main__':
    # --- 1. 단층 퍼셉트론 (SLP)으로 AND 학습 예제 ---
    print("--- 단층 퍼셉트론 (SLP)으로 AND 게이트 학습 ---")
    X_and = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y_and = np.array([[0], [0], [0], [1]])
    X = X_and
    y = y_and

    # print("--- 단층 퍼셉트론 (SLP)으로 XOR 게이트 학습 ---")
    # X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    # y_xor = np.array([[0], [1], [1], [0]])
    # X = X_xor
    # y = y_xor

    # 모델 파라미터
    input_dim = X.shape[1]     # 2
    output_dim = y.shape[1]    # 1
    learning_rate_slp = 0.1
    epochs_slp = 1000  # 학습횟수를 늘리거나 줄여본다.

    # SLP 모델 생성 및 학습
    slp_model = SLP(input_size=input_dim, output_size=output_dim, seed=123)
    print(f"초기 가중치(W):\n{slp_model.W}")
    print(f"초기 편향(b): {slp_model.b}\n")

    print("--모델 학습--")
    training_losses_slp = slp_model.train(X, y, epochs_slp, learning_rate_slp, verbose_interval=epochs_slp // 10)
    print(f"\n학습 후 가중치(W):\n{slp_model.W}")
    print(f"학습 후 편향(b): {slp_model.b}\n")

    # 학습 후 SLP 예측 결과 출력
    print("\n학습 후 SLP 예측 결과:")
    predictions_slp = slp_model.predict(X)
    for i in range(len(X)):
        predicted_class_slp = 1 if predictions_slp[i][0] >= 0.5 else 0
        print(f"Input: {X[i]}, True Output: {y[i][0]}, Predicted Output prob: {predictions_slp[i][0]:.4f} (Class: {predicted_class_slp})")

    print("-" * 50)



---



## **2.심층신경망 : 다층퍼셉트론(MLP)**

- **다층 퍼셉트론(Multi-Layer Perceptron, MLP)**
    - 입력층(input layer)과 출력층(output layer) 사이에 하나 이상의 은닉층(hidden layer)을 추가한 신경망 구조
    - 각 층은 여러 개의 뉴런(노드)으로 구성되며, 층간 뉴런들은 완전 연결(fully connected)되어 있음
    - 은닉층과 비선형 활성화 함수를 사용함으로써 MLP는 비선형 문제도 해결할 수 있게 됨

### **1) XOR 게이트**

In [ ]:
# -----------------
# 다층 퍼셉트론 (XOR 추가)
# -----------------
def XOR(x1, x2):
    """
    XOR는 단층 퍼셉트론으로 구현할 수 없으므로,
    기존의 NAND, OR, AND 게이트를 조합하여 다층 구조로 구현합니다.
    Layer 1: NAND, OR
    Layer 2: AND
    """
    s1 = NAND(x1, x2)
    s2 = OR(x1, x2)
    y = AND(s1, s2)
    return y

# 입력 조합
X = [(0, 0), (0, 1), (1, 0), (1, 1)]

# 출력 확인 (XOR 열 추가)
print("x1 x2 | AND OR NAND NOR | XOR")
print("-------------------------------")
for x1, x2 in X:
    print(f" {x1}  {x2} |  {AND(x1,x2)}   {OR(x1,x2)}   {NAND(x1,x2)}    {NOR(x1,x2)}  |  {XOR(x1,x2)}")


### **2) 다층 퍼셉트론** (XOR 게이트 예측 : 이진분류)


In [ ]:
import numpy as np

# --- 1. 활성화 함수 및 손실 함수 ---
def sigmoid(x):
    """시그모이드 함수"""
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    """시그모이드 함수의 도함수"""
    # 참고: 입력 x는 이미 시그모이드 함수를 통과한 출력값(a)이라고 가정
    return x * (1 - x)

def softmax(x):
    """소프트맥스 함수"""
    # 오버플로우 방지를 위해 입력값에서 최댓값을 빼줌
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def mse_loss(y_true, y_pred):
    """평균 제곱 오차 (MSE) 손실 함수"""
    return np.mean((y_true - y_pred) ** 2)

def cross_entropy_loss(y_true, y_pred):
    """교차 엔트로피 손실 함수 (y_true는 원-핫 인코딩)"""
    num_samples = y_true.shape[0]
    # 아주 작은 값(epsilon)을 더해 log(0) 방지
    epsilon = 1e-12
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)
    loss = -np.sum(y_true * np.log(y_pred)) / num_samples
    return loss


# --- 2.다층 퍼셉트론 (MLP) 클래스 ---
class MLP:
    def __init__(self, input_size, hidden_size, output_size, output_activation='sigmoid', seed=None):
        """
        MLP 모델 초기화
        input_size: 입력층 노드 수
        hidden_size: 은닉층 노드 수
        output_size: 출력층 노드 수
        seed: 가중치 초기화를 위한 랜덤 시드 (재현성을 위해)
        """
        if seed is not None: np.random.seed(seed)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.output_activation = output_activation.lower() # 'sigmoid' 또는 'softmax'

        # 가중치 초기화
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size) # He 초기화 유사
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size) # He 초기화 유사
        self.b2 = np.zeros((1, output_size))

        self.z1, self.a1, self.z2, self.a2 = None, None, None, None

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = sigmoid(self.z1) # 은닉층은 시그모이드 고정 (ReLU 등으로 변경 가능)

        self.z2 = np.dot(self.a1, self.W2) + self.b2
        if self.output_activation == 'softmax':      # 다중분류
            self.a2 = softmax(self.z2)
        elif self.output_activation == 'sigmoid':    # 이진분류
            self.a2 = sigmoid(self.z2)
        else:                                        # 시그모이드(기본값)
            self.a2 = sigmoid(self.z2)
        return self.a2

    def backward(self, X, y_true, learning_rate):
        num_samples = X.shape[0]

        # 출력층 델타 계산
        if self.output_activation == 'softmax':
            # y_true는 원-핫 인코딩된 형태여야 함
            # 소프트맥스 + 교차 엔트로피 손실의 경우, 출력층 델타는 (예측값 - 실제값)
            delta2 = self.a2 - y_true
        elif self.output_activation == 'sigmoid':
            # 시그모이드 + MSE 손실의 경우
            delta2 = (self.a2 - y_true) * sigmoid_derivative(self.a2)
        else: # 기본 (시그모이드 + MSE)
            delta2 = (self.a2 - y_true) * sigmoid_derivative(self.a2)

        # 은닉층 델타 계산
        error_hidden_layer = np.dot(delta2, self.W2.T)
        delta1 = error_hidden_layer * sigmoid_derivative(self.a1)

        # 기울기 계산
        dW2 = (1/num_samples) * np.dot(self.a1.T, delta2)
        db2 = (1/num_samples) * np.sum(delta2, axis=0, keepdims=True)
        dW1 = (1/num_samples) * np.dot(X.T, delta1)
        db1 = (1/num_samples) * np.sum(delta1, axis=0, keepdims=True)

        # 가중치 및 편향 업데이트
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1

    def train(self, X, y, epochs, learning_rate, verbose_interval=None):
        losses = []
        # y가 원-핫 인코딩이 필요한 경우 (softmax 사용 시) train 외부에서 처리하거나, 여기서 확인
        # 여기서는 y가 이미 적절한 형태로 들어온다고 가정

        for epoch in range(epochs):
            y_pred = self.forward(X)

            if self.output_activation == 'softmax':
                loss = cross_entropy_loss(y, y_pred) # y는 원-핫 인코딩된 형태
            elif self.output_activation == 'sigmoid':
                loss = mse_loss(y, y_pred) # y는 일반적인 타겟 값
            else: # 기본 (시그모이드 + MSE)
                loss = mse_loss(y, y_pred)
            losses.append(loss)

            self.backward(X, y, learning_rate)

            if verbose_interval and (epoch + 1) % verbose_interval == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.6f}")

        if verbose_interval and epochs > 0 and epochs % verbose_interval != 0:
             print(f"Epoch {epochs}/{epochs}, Loss: {losses[-1]:.6f}")
        elif not verbose_interval and epochs > 0 :
             print(f"Epoch {epochs}/{epochs}, Loss: {losses[-1]:.6f}")
        return losses

    def predict_proba(self, X):
        """클래스별 확률 예측 (주로 softmax와 함께 사용)"""
        return self.forward(X)

    def predict(self, X):
        """최종 클래스 예측"""
        probabilities = self.forward(X)
        if self.output_activation == 'softmax':
            return np.argmax(probabilities, axis=1) # 가장 높은 확률의 클래스 인덱스
        elif self.output_activation == 'sigmoid':
            return (probabilities > 0.5).astype(int) # 이진 분류 임계값
        else: # 기본
            return (probabilities > 0.5).astype(int)


In [ ]:
# --- 메인 실행 부분 ---
if __name__ == '__main__':
    # --- 2. 다층 퍼셉트론 (MLP)으로 XOR 게이트 학습 예제 ---
    print("\n--- 다층 퍼셉트론 (MLP)으로 XOR 게이트 학습 (Sigmoid 출력) ---")
    X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y_xor = np.array([[0], [1], [1], [0]])

    # 모델 파라미터
    input_dim_xor = X_xor.shape[1]   # 2
    hidden_dim_mlp_xor = 4           # 4
    output_dim_xor = y_xor.shape[1]  # 1
    learning_rate_mlp_xor = 0.1
    epochs_mlp_xor = 20000

    # MLP 모델 생성
    mlp_model = MLP(input_size=input_dim_xor, hidden_size=hidden_dim_mlp_xor, output_size=output_dim_xor, seed=42)
    print("--- 초기 가중치 및 편향 ---")
    print(f"초기 가중치(W1):\n{mlp_model.W1}")
    print(f"\n초기 가중치(W2):\n{mlp_model.W2}")
    print(f"\n초기 편향(b1): {mlp_model.b1}")
    print(f"초기 편향(b2): {mlp_model.b2}\n")

    # 모델 학습
    # train() 메서드는 loss 리스트를 반환합니다.
    # 이 값을 mlp_model 변수에 다시 할당하면 모델 객체가 덮어쓰여지므로,
    # 반환값은 다른 변수(mlp_losses)에 저장합니다.
    # train() 메서드가 실행되면 mlp_model 객체의 가중치는 내부적으로 업데이트됩니다.
    print("--- 모델 학습 시작 ---")
    mlp_losses = mlp_model.train(X_xor, y_xor, epochs_mlp_xor, learning_rate_mlp_xor, verbose_interval=epochs_mlp_xor // 10)
    print("--- 모델 학습 완료 ---\n")

    # 요청하신 학습 후 가중치 및 편향 출력 부분입니다.
    print("--- 학습 후 가중치 및 편향 ---")
    print(f"학습 후 가중치(W1):\n{mlp_model.W1}")
    print(f"\n학습 후 가중치(W2):\n{mlp_model.W2}")
    print(f"\n학습 후 편향(b1): {mlp_model.b1}")
    print(f"학습 후 편향(b2): {mlp_model.b2}\n")


    # 학습 후 MLP 예측 결과 출력 (XOR 게이트)
    # predict()는 최종 클래스(0 또는 1)를, predict_proba()는 확률을 반환합니다.
    # 두 가지 모두 출력하여 결과를 더 명확하게 확인합니다.
    print("--- XOR 게이트 학습 후 MLP 예측 결과 ---")
    probabilities = mlp_model.predict_proba(X_xor)
    predictions = mlp_model.predict(X_xor)
    for i in range(len(X_xor)):
        print(f"Input: {X_xor[i]}, "
              f"True Output: {y_xor[i][0]}, "
              f"Predicted Probability: {probabilities[i][0]:.4f}, "
              f"Predicted Class: {predictions[i][0]}")


In [ ]:
# 손실 그래프 시각화
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].plot(training_losses_slp)
ax[0].set_title('SLP Training Loss for AND Gate')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('MSE Loss')
ax[0].grid(True)

ax[1].plot(training_losses_mlp)
ax[1].set_title('MLP Training Loss for XOR Gate')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('MSE Loss')
ax[1].grid(True)

plt.tight_layout()
plt.show()


### **3) 다층 퍼셉트론 (iris 예측 : 다중분류)**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score

print("\n--- 다층 퍼셉트론 (MLP)으로 Iris 데이터셋 학습 (Softmax 출력) ---")
# 데이터 로드
iris = load_iris()
X_iris = iris.data
y_iris_original = iris.target.reshape(-1, 1) # (n_samples, 1) 형태로

# 데이터 전처리
# 1. 입력 데이터 스케일링
scaler = StandardScaler()
X_iris_scaled = scaler.fit_transform(X_iris)

# 2. 타겟 데이터 원-핫 인코딩
encoder = OneHotEncoder(sparse_output=False, categories='auto') # sparse=False -> dense array
y_iris_one_hot = encoder.fit_transform(y_iris_original)
# y_iris_one_hot의 shape: (150, 3)

# 3. 학습/테스트 데이터 분리
X_train_iris, X_test_iris, y_train_iris_one_hot, y_test_iris_one_hot = train_test_split(
    X_iris_scaled, y_iris_one_hot, test_size=0.2, random_state=42, stratify=y_iris_original
)
# 테스트용 원본 레이블 (비교용)
_, _, y_train_iris_orig, y_test_iris_orig = train_test_split(
    X_iris_scaled, y_iris_original, test_size=0.2, random_state=42, stratify=y_iris_original
)


# MLP 모델 파라미터 (Iris)
input_dim_iris = X_train_iris.shape[1]          # 4
hidden_dim_mlp_iris = 8                         # 8 은닉층 노드 수 (조정 가능)
output_dim_iris = y_train_iris_one_hot.shape[1] # 3 (클래스 개수)
learning_rate_mlp_iris = 0.01                   # 학습률 (조정 가능)
epochs_mlp_iris = 1000                          # 에포크 수 (조정 가능)

# MLP 모델 생성 (출력 활성화: softmax)
mlp_model_iris = MLP(input_size=input_dim_iris,
                        hidden_size=hidden_dim_mlp_iris,
                        output_size=output_dim_iris,
                        output_activation='softmax',
                        seed=42)

# 모델 학습
print("Iris 데이터셋 학습 시작...")
mlp_model_iris.train(X_train_iris, y_train_iris_one_hot, epochs_mlp_iris, learning_rate_mlp_iris, verbose_interval=epochs_mlp_iris//10)

# 테스트 데이터로 예측 및 평가
y_pred_iris_proba = mlp_model_iris.predict_proba(X_test_iris) # 확률
y_pred_iris_classes = mlp_model_iris.predict(X_test_iris)    # 클래스 레이블

# 정확도 계산
# y_test_iris_one_hot의 argmax는 원본 레이블과 동일
accuracy = accuracy_score(np.argmax(y_test_iris_one_hot, axis=1), y_pred_iris_classes)
# 또는 accuracy_score(y_test_iris_orig.flatten(), y_pred_iris_classes)

print(f"\nIris 데이터셋 테스트 정확도: {accuracy*100:.2f}%")

# 일부 예측 결과 출력
print("\nIris 테스트 데이터 일부 예측 결과 (첫 5개 샘플):")
for i in range(min(5, X_test_iris.shape[0])):
    true_class = iris.target_names[y_test_iris_orig[i][0]]
    predicted_class_name = iris.target_names[y_pred_iris_classes[i]]
    print(f"샘플 {i+1}: 실제 클래스 = {true_class:<10}, 예측 클래스 = {predicted_class_name:<10}, 확률 = {y_pred_iris_proba[i]}")





---

